***First time using R with Jupyter; first time using R in months... This should be fun...***

**Reference:** https://stackoverflow.com/questions/27697504/ocean-latitude-longitude-point-distance-from-shore

In [ ]:
#' @importFrom methods setClass setGeneric setMethod setRefClass

In [ ]:
# is.installed  <- function(mypkg) is.element(mypkg, installed.packages()[,1])
# is.installed('rgdal')

In [ ]:
# install.packages('installr', 'C:/Users/Laolu/Anaconda2/envs/py27/R/library', repos='http://cran.rstudio.com/')

In [1]:
version # trying to update R from 3.3.2 to 3.3.3 to ensure gdal package works perfectly 

               _                           
platform       x86_64-w64-mingw32          
arch           x86_64                      
os             mingw32                     
system         x86_64, mingw32             
status                                     
major          3                           
minor          3.2                         
year           2016                        
month          10                          
day            31                          
svn rev        71607                       
language       R                           
version.string R version 3.3.2 (2016-10-31)
nickname       Sincere Pumpkin Patch       

In [2]:
# setInternet2(TRUE)
# installr::updateR() # doesn't work

In [2]:
find.package('rgeos')
find.package('rgdal')

[1] "C:/Users/Laolu/Documents/R/win-library/3.3/rgeos"

[1] "C:/Users/Laolu/Documents/R/win-library/3.3/rgdal"

In [ ]:
# install.packages('rgeos', 'C:/Users/Laolu/Anaconda2/envs/py27/R/library', repos='http://cran.rstudio.com/')
# install.packages('rgdal', 'C:/Users/Laolu/Anaconda2/envs/py27/R/library', repos='http://cran.rstudio.com/')

In [3]:
df  <- read.csv("data.csv")
head(df)

address,area,baths,beds,buildingtype,latitude,longitude,neighborhood,price,borough,pricearea
56 Walker Street #2,2670,3.5,3,Condo,40.7190,-74.0032,Tribeca,5600000,manhattan,2097.378
201 East 62nd Street #5A,2240,3.5,3,Co-op,40.7635,-73.9648,Lenox Hill,2595000,manhattan,1158.482
333 West 56th Street #2,820,1.0,1,Condo,40.7670,-73.9849,Hell's Kitchen,995000,manhattan,1213.415
330 East 38th Street #20P,884,1.5,1,Condo,40.7463,-73.9727,Murray Hill,1225000,manhattan,1385.747
322 West 57th Street #55U,1599,2.0,2,Condo,40.7668,-73.9842,Hell's Kitchen,4595000,manhattan,2873.671
408 Eighth Avenue #3C,712,1.0,1,Condo,40.7498,-73.9948,Midtown South,1099000,manhattan,1543.539


In [4]:
library(dplyr)

Warning message:
"package 'dplyr' was built under R version 3.3.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [5]:
df %>% group_by(borough) %>% summarise(avg_price = mean(price), max_price = max(price), min_price = min(price), total = n())

borough,avg_price,max_price,min_price,total
bronx,708825.5,6999000,44500,753
brooklyn,1458346.3,11950000,128500,4317
manhattan,4230556.9,85000000,160000,7770
queens,881813.9,5300000,110000,2237
staten_island,813081.1,2399000,225000,221


In [6]:
library(rgeos)
library(rgdal)

Warning message:
"package 'rgeos' was built under R version 3.3.3"rgeos version: 0.3-23, (SVN revision 546)
 GEOS runtime version: 3.5.0-CAPI-1.9.0 r4084 
 Linking to sp version: 1.2-5 
 Polygon checking: TRUE 

Warning message:
"package 'rgdal' was built under R version 3.3.3"Loading required package: sp
Warning message:
"package 'sp' was built under R version 3.3.3"rgdal: version: 1.2-8, (SVN revision 663)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.0.1, released 2015/09/15
 Path to GDAL shared files: C:/Users/Laolu/Documents/R/win-library/3.3/rgdal/gdal
 Loaded PROJ.4 runtime: Rel. 4.9.2, 08 September 2015, [PJ_VERSION: 492]
 Path to PROJ.4 shared files: C:/Users/Laolu/Documents/R/win-library/3.3/rgdal/proj
 Linking to sp version: 1.2-5 


In [7]:
wgs.84  <- "+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0"
mollweide  <- "+proj=moll +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"

In [8]:
# fixed the anomaly by passing long before lat instead of c('latitude', 'longitude') as done previously
points  <- SpatialPoints(df[c('longitude', 'latitude')], proj4string = CRS(wgs.84))

In [9]:
coast  <- readOGR(dsn="C:/Users/Laolu/Dropbox/GitHub/NYC_RealEstate/coastline_data",layer="ne_10m_coastline", p4s=wgs.84)

OGR data source with driver: ESRI Shapefile 
Source: "C:/Users/Laolu/Dropbox/GitHub/NYC_RealEstate/coastline_data", layer: "ne_10m_coastline"
with 4132 features
It has 2 fields


Warning message in readOGR(dsn = "C:/Users/Laolu/Dropbox/GitHub/NYC_RealEstate/coastline_data", :
"p4s= argument given as: +proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0
 and read as: +proj=longlat +datum=WGS84 +no_defs 
 read string overridden by given p4s= argument value"

In [10]:
typeof(coast)
typeof(points)

[1] "S4"

[1] "S4"

In [11]:
class(points)
class(coast)

[1] "SpatialLinesDataFrame"
attr(,"package")
[1] "sp"

In [12]:
coast.moll  <- spTransform(coast, CRS(mollweide))
points.moll  <- spTransform(points, CRS(mollweide))

In [12]:
getSlots("SpatialPoints")
getSlots("SpatialLinesDataFrame")

coords        bbox proj4string 
   "matrix"    "matrix"       "CRS"

data        lines         bbox  proj4string 
"data.frame"       "list"     "matrix"        "CRS"

In [13]:
# coastdistance  <- sapply(points, function(i)gDistance(points.moll[i], coast.moll), simplify = "array")
len  <- c(1:length(points))
coastdistance  <- sapply(len, function(i){gDistance(points.moll[i], coast.moll)})

In [14]:
df$coastdist <- coastdistance
df

address,area,baths,beds,buildingtype,latitude,longitude,neighborhood,price,borough,pricearea,coastdist
56 Walker Street #2,2670,3.5,3,Condo,40.71900,-74.00320,Tribeca,5600000,manhattan,2097.378,746.61028
201 East 62nd Street #5A,2240,3.5,3,Co-op,40.76350,-73.96480,Lenox Hill,2595000,manhattan,1158.482,501.93385
333 West 56th Street #2,820,1.0,1,Condo,40.76700,-73.98490,Hell's Kitchen,995000,manhattan,1213.415,598.44922
330 East 38th Street #20P,884,1.5,1,Condo,40.74630,-73.97270,Murray Hill,1225000,manhattan,1385.747,257.25204
322 West 57th Street #55U,1599,2.0,2,Condo,40.76680,-73.98420,Hell's Kitchen,4595000,manhattan,2873.671,647.07872
408 Eighth Avenue #3C,712,1.0,1,Condo,40.74980,-73.99480,Midtown South,1099000,manhattan,1543.539,691.42652
151 East 85th Street #12H,1646,2.5,2,Condo,40.77910,-73.95540,Upper East Side,3095000,manhattan,1880.316,594.59617
30 Park Place #48B,1664,2.5,2,Condo,40.71278,-74.00903,Tribeca,4520000,manhattan,2716.346,404.38246
438 East 12th Street #5R,1604,3.0,3,Condo,40.72900,-73.98210,East Village,3450000,manhattan,2150.873,701.18010
80 Riverside Boulevard #9P,1850,3.0,3,Condo,40.77550,-73.98850,Lincoln Square,3695000,manhattan,1997.297,42.81556


In [15]:
write.csv(df, file = "data2.csv", row.names=FALSE)

In [17]:
# getSlots("SpatialPolygonsDataFrame")

data     polygons    plotOrder         bbox  proj4string 
"data.frame"       "list"    "integer"     "matrix"        "CRS"